<a href="https://colab.research.google.com/github/Ry02024/chatbot_streamlit/blob/main/chatbot_streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 初期設定

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 環境構築

In [3]:
!pip install streamlit -q
!pip install pyngrok -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.1 MB/s eta 0:00:00


In [6]:
import streamlit as st
from pyngrok import ngrok

In [7]:
from google.colab import userdata
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')

In [8]:
from pyngrok import ngrok
import os

# ngrokトークンを設定
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [ ]:
import os

# Streamlit設定ファイルのディレクトリを作成
os.makedirs('/root/.streamlit', exist_ok=True)

# config.tomlファイルを作成して最大アップロードサイズを設定
with open('/root/.streamlit/config.toml', 'w') as f:
    f.write("""
[server]
maxUploadSize = 1024
""")

# 実行ファイル

In [37]:
%%writefile streamlit_app.py
import streamlit as st
import google.generativeai as genai

# タイトルと説明の表示
st.title("💬 Gemini Chatbot")
st.write(
    "This is a chatbot using Google's Gemini API. Please provide your API key to start."
)

# Gemini APIキーの入力
gemini_api_key = st.text_input("Gemini API Key", type="password")

# セッションの初期化
if "messages" not in st.session_state:
    st.session_state.messages = []

# APIキーが入力された場合の処理
if gemini_api_key:
    try:
        # Geminiクライアントの設定
        genai.configure(api_key=gemini_api_key)

        # モデルの定義
        model = genai.GenerativeModel(model_name='gemini-1.5-pro')

        # チャット履歴の表示
        st.subheader("Chat History")
        for message in st.session_state.messages:
            with st.chat_message(message["role"]):
                st.markdown(message["content"])

        # ユーザー入力
        prompt = st.chat_input("Enter your message:")
        if prompt:
            # ユーザーのメッセージを保存
            st.session_state.messages.append({"role": "user", "content": prompt})
            with st.chat_message("user"):
                st.markdown(prompt)

            # Gemini APIを使用して応答を生成
            try:
                response = model.generate_content(contents=[prompt])
                assistant_reply = response.text  # 応答を取得

                # 応答を保存
                st.session_state.messages.append({"role": "assistant", "content": assistant_reply})

                # 応答の表示
                with st.chat_message("assistant"):
                    st.markdown(assistant_reply)

            except Exception as e:
                st.error(f"Error occurred while generating response: {e}")

    except Exception as e:
        st.error(f"Failed to configure the Gemini API: {e}")
else:
    st.info("Please provide your Gemini API key to start the chatbot.", icon="🗝️")


Overwriting streamlit_app.py


#アプリの起動

In [38]:
# ngrokを介してStreamlitを公開
public_url = ngrok.connect(addr='8501')
print('Public URL:', public_url)

Public URL: NgrokTunnel: "https://ef7a-34-85-175-190.ngrok-free.app" -> "http://localhost:8501"


In [39]:
# Streamlitアプリケーションの起動
!streamlit run streamlit_app.py >/dev/null